In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image, ImageOps
from DL import DLLayer, DLModel
from sklearn.metrics import confusion_matrix

In [2]:
data = pd.read_csv('fer2013.csv')
data.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [3]:
emotions_names = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}
data['emotion_name'] = data['emotion'].map(emotions_names)
data.head()

,emotion,pixels,Usage,emotion_name
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training,Angry
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training,Angry
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training,Fear
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training,Sad
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training,Neutral


In [4]:
pixels_values = data.pixels.str.split(" ").tolist()
pixels_values = pd.DataFrame(pixels_values, dtype=np.int8)

C:\Users\User\anaconda3\envs\myenv\lib\site-packages\IPython\core\interactiveshell.py:3441: FutureWarning: Could not cast to int8, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
images = pixels_values.values
images = images.astype(float)

In [6]:
print(images.shape)

(35887, 2304)


In [7]:
image_pixels = images.shape[1]
image_pixels

2304

In [8]:
labels_flat = data["emotion"].values.ravel()
labels_count = np.unique(labels_flat).shape[0]

In [9]:
def dense_to_one_hot(labels_dense, num_classes):
    num_labels = labels_dense.shape[0]
    index_offset = np.arange(num_labels) * num_classes
    labels_one_hot = np.zeros((num_labels, num_classes))
    labels_one_hot.flat[[index_offset + labels_dense.ravel()]] = 1
    return labels_one_hot

In [10]:
labels = dense_to_one_hot(labels_flat, labels_count)
labels = labels.astype(np.uint8)
print(labels)

[[1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]]


In [11]:
m = 32298
X_train, Y_train = images[:m].T, labels[:m].T
X_test, Y_test = images[m:].T, labels[m:].T
print(X_train.shape, Y_train.shape)

(2304, 32298) (7, 32298)


In [37]:
model = DLModel()
model.add(DLLayer("Hidden 1", 180, (2304,), "sigmoid", learning_rate=0.01))
model.add(DLLayer("Hidden 1", 64, (180,), "relu", learning_rate=0.01))
model.add(DLLayer("Output", 7, (64,), "softmax", learning_rate=0.01))

In [38]:
model.compile("categorical_cross_entropy")

In [39]:
costs = model.train(X_train, Y_train, 1500)

cost after  1 %: 1.940366565362037
cost after  2 %: 1.9335284985529464
cost after  3 %: 1.9264391534154848
cost after  4 %: 1.918468313688876
cost after  5 %: 1.9091263507536127
cost after  6 %: 1.8982903201289554
cost after  7 %: 1.8860333429271512
cost after  8 %: 1.8727865451238916
cost after  9 %: 1.8596601600894618
cost after  10 %: 1.8481817819103437
cost after  11 %: 1.8390112572009194
cost after  12 %: 1.832131869026907
cost after  13 %: 1.827036434087206
cost after  14 %: 1.82320428364772
cost after  15 %: 1.8202511363929248
cost after  16 %: 1.8179696961395222
cost after  17 %: 1.8161851947609107
cost after  18 %: 1.8147827564571326
cost after  19 %: 1.8136728645146545
cost after  20 %: 1.8127883964861318
cost after  21 %: 1.8120812850567045
cost after  22 %: 1.8115060034385828
cost after  23 %: 1.8110409866668822
cost after  24 %: 1.8106631024320723
cost after  25 %: 1.8103588386906435
cost after  26 %: 1.8101034111974181
cost after  27 %: 1.8098924262130285
cost after  28 %

In [40]:
model.save_weights('weights3')

plt.plot(np.squeeze(costs))
plt.ylabel('cost')
plt.xlabel('iterations')
plt.show()

In [41]:
def predict_softmax(X, Y, model):
    AL = model.predict(X)
    predictions = np.argmax(AL, axis=0)
    labels = np.argmax(Y, axis=0)
    return confusion_matrix(predictions, labels)

In [42]:
print('Deep train accuracy')
pred_train = predict_softmax(X_train, Y_train, model)
print(pred_train)

Deep train accuracy
[[   0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0]
 [4462  492 4593 8110 5483 3586 5572]
 [   0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0]]


In [43]:
print('Deep test accuracy')
pred_test = predict_softmax(X_test, Y_test, model)
print(pred_test)

Deep test accuracy
[[  0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0]
 [491  55 528 879 594 416 626]
 [  0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0]]


In [44]:
i = 4
print('train', str(i), str(pred_train[i][i] / np.sum(pred_train[:, i])))
print('test', str(i), str(pred_test[i][i] / np.sum(pred_test[:, i])))

train 4 0.0
test 4 0.0
